# IRCサーバーのテストケース

## 基本チェック (Basic checks)

1. **コンパイルとMakefile**
   ```bash
   make clean
   make
   ls -la ircserv # 確認
   ```

2. **単一poll()/select()確認**
   ```bash
   grep -r "poll\|select\|epoll\|kqueue" src/ 
   # 結果が1つだけの場所を示すはず
   ```

3. **非ブロッキングI/O設定の確認**
   ```bash
   grep -r "fcntl" src/
   # 各fcntlがO_NONBLOCKの設定のみか確認
   ```

4. **poll()前のI/O操作がないか確認**
   ```bash
   # ソースコードを確認し、accept/recv/sendの前に必ずselect/pollが呼ばれているか
   # errno == EAGAINによる再読み取りがないか確認
   ```

## ネットワーキング (Networking)

1. **サーバー起動とリスニング**
   ```bash
   ./ircserv 6667 testpass
   # 別ターミナルで
   netstat -tuln | grep 6667
   # 0.0.0.0:6667 と表示されるはず
   ```

2. **基本接続テスト**
   ```bash
   nc localhost 6667
   # パスワード入力後:
   NICK test1
   USER test1 * * :Test User
   JOIN #test
   PRIVMSG #test :Hello world
   ```

3. **複数クライアント同時接続**
   ```
   # 3つの異なるターミナルでncを実行し同時に操作
   # 全てに正しく応答があるか確認
   ```

4. **リファレンスIRCクライアントテスト**
   ```
   # チームが使用するIRCクライアントで接続し
   # 基本操作（認証、ニックネーム設定、チャンネル参加）ができるか確認
   ```

## ネットワーク特殊ケース (Networking specials)

1. **部分コマンド送信テスト**
   ```bash
   nc localhost 6667
   # パスワード認証後:
   com^D  # Ctrl+D で送信を区切る
   man^D
   d^D
   ```

2. **クライアント強制終了テスト**
   ```bash
   # 複数クライアント接続中に、1つのターミナルをkill
   # 他のクライアントが引き続き動作するか確認
   ```

3. **部分送信中の切断テスト**
   ```bash
   nc localhost 6667
   # パスワード認証後:
   PRIVMSG #test :This is a lo^C  # 途中でCtrl+C
   # 他のクライアントが正常に動作するか確認
   ```

4. **クライアント一時停止とフラッド**
   ```bash
   # クライアント1をCtrl+Zで一時停止
   # クライアント2から多数のメッセージを連続送信
   # サーバーがハングしないことを確認
   # クライアント1をfgで再開し、滞留メッセージが処理されるか確認
   ```

## クライアントコマンド基本 (Client Commands basic)

1. **認証とチャンネル操作**
   ```
   # 前述のテストで実施済み
   ```

2. **PRIVMSG詳細テスト**
   ```bash
   # ユーザー間プライベートメッセージ
   PRIVMSG test2 :Hello, this is a private message
   
   # チャンネルメッセージ
   PRIVMSG #test :Channel broadcast test
   
   # 特殊文字含むメッセージ
   PRIVMSG #test :Test with special chars: !@#$%^&*()_+-=[]{}|;':",./<>?
   ```

## チャンネルオペレータコマンド (Client Commands channel operator)

1. **一般ユーザー制限テスト**
   ```bash
   # 非オペレータでオペレータコマンドを試行
   KICK #test test2
   MODE #test +i
   TOPIC #test :Unauthorized topic
   # すべて拒否されるはず
   ```

2. **オペレータコマンドテスト**
   ```bash
   # チャンネル作成者でテスト
   KICK #test test2 :Testing kick command
   INVITE test2 #test
   TOPIC #test :New channel topic
   ```

3. **MODEコマンドテスト**
   ```bash
   # 各モードを設定・解除してテスト
   MODE #test +i  # 招待制に変更
   # 招待なしで参加できないことを確認
   
   MODE #test +t  # トピック制限
   # 一般ユーザーがトピック変更できないことを確認
   
   MODE #test +k password  # パスワード設定
   # パスワードなしで参加できないことを確認
   
   MODE #test +o test2  # オペレータ権限付与
   # test2がオペレータ機能を使えることを確認
   
   MODE #test +l 5  # ユーザー数制限
   # 6人目が参加できないことを確認
   ```

## 問題発生時の追加確認

1. **メモリリーク確認**
   ```bash
   valgrind --leak-check=full ./ircserv 6667 testpass
   # メモリリークがないかチェック
   ```

2. **エッジケース対応確認**
   ```bash
   # 存在しないコマンドを送信
   NOTACOMMAND
   
   # 存在しないユーザー・チャンネルへのメッセージ
   PRIVMSG #nonexistent :Test
   PRIVMSG nonuser :Test
   ```

これらのテストケースを実行することで、IRCサーバーが課題要件に沿って正しく動作するか検証できます。